In [3]:
# Loaded variable 'df' from URI: c:\Users\hp\IA\data2.csv
import pandas as pd
df = pd.read_csv(r'c:\Users\hp\IA\data2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            1319 non-null   int64  
 1   gender         1319 non-null   int64  
 2   status         1319 non-null   object 
 3   pressurehight  1319 non-null   int64  
 4   pressurelow    1319 non-null   int64  
 5   glucose        1319 non-null   float64
 6   kcm            1319 non-null   float64
 7   troponin       1319 non-null   float64
 8   impluse        1319 non-null   int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 92.9+ KB


In [4]:
df.describe()


,age,gender,pressurehight,pressurelow,glucose,kcm,troponin,impluse
count,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000,1319.000000
mean,56.191812,0.659591,127.170584,72.269143,146.634344,15.274306,0.360942,78.336619
std,13.647315,0.474027,26.122720,14.033924,74.923045,46.327083,1.154568,51.630270
min,14.000000,0.000000,42.000000,38.000000,35.000000,0.321000,0.001000,20.000000
25%,47.000000,0.000000,110.000000,62.000000,98.000000,1.655000,0.006000,64.000000
50%,58.000000,1.000000,124.000000,72.000000,116.000000,2.850000,0.014000,74.000000
75%,65.000000,1.000000,143.000000,81.000000,169.500000,5.805000,0.085500,85.000000
max,103.000000,1.000000,223.000000,154.000000,541.000000,300.000000,10.300000,1111.000000


In [5]:
df.isnull().sum()

age              0
gender           0
status           0
pressurehight    0
pressurelow      0
glucose          0
kcm              0
troponin         0
impluse          0
dtype: int64

In [6]:
def clean_data(df):
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    # Drop rows with missing data in column: 'age'
    df = df.dropna(subset=['age'])
    return df


In [7]:
#check for duplicates 
print(df.duplicated().any())

False


In [13]:
# data preparation 
X=df.drop(columns=['status'])
y=df['status']

In [15]:
from sklearn.model_selection import train_test_split
#Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
num_cols = df.select_dtypes(include='number').columns
print(num_cols)

Index(['age', 'gender', 'pressurehight', 'pressurelow', 'glucose', 'kcm',
       'troponin', 'impluse'],
      dtype='object')


In [19]:
# normalisation
from sklearn.preprocessing import StandardScaler
num_cols=['age','pressurehight','pressurelow','glucose','kcm','troponin', 'impluse']
scale=StandardScaler()
X_train[num_cols]=scale.fit_transform(X_train[num_cols])
X_test[num_cols]=scale.transform(X_test[num_cols])
data_scaled=X_train[num_cols].copy()
print(X_train)

           age  gender  pressurehight  pressurelow   glucose       kcm  \
598  -1.034860       1       1.012203     1.322291 -0.651508 -0.213444   
1213  1.462646       1      -0.617725    -0.234964 -0.730903 -0.277728   
209   0.287349       1      -3.295463    -0.588885 -0.545649 -0.308747   
538  -0.300299       0       0.119624     0.543664  0.446783 -0.286466   
140  -0.814492       0      -1.549112    -1.792218 -0.863227 -0.277520   
...        ...     ...            ...          ...       ...       ...   
1095 -0.447212       0      -0.307262     0.260526 -0.228071  0.007911   
1130  0.066981       1      -1.005803    -1.296728 -0.426557 -0.309850   
1294 -0.447212       0      -1.355073    -1.225944  3.543170  0.025595   
860  -0.153387       1       1.051011     0.826801 -0.267768 -0.265246   
1126  3.445959       0       1.710744    -1.155159  0.036578  0.753944   

      troponin   impluse  
598  -0.310036  0.101709  
1213 -0.284726  1.175163  
209   0.072336 -0.214012  
538

In [ ]:
import pandas as pd

def clean_data(df):
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    # Drop rows with missing data in column: 'age'
    df = df.dropna(subset=['age'])
    # Drop rows with missing data across all columns
    df = df.dropna()
    # Replace missing values with 0 in columns: 'age', 'gender' and 3 other columns
    df = df.fillna({'age': 0, 'gender': 0, 'pressurehight': 0, 'pressurelow': 0, 'impluse': 0})
    # Multi-label encode column 'status' using delimiter 'status'
    loc_0 = df.columns.get_loc('status')
    df_encoded = df['status'].str.get_dummies(sep='status').add_prefix('status_')
    df = pd.concat([df.iloc[:,:loc_0], df_encoded, df.iloc[:,loc_0+1:]], axis=1)
    return df

# Loaded variable 'df' from URI: c:\Users\hp\IA\data2.csv
df = pd.read_csv(r'c:\Users\hp\IA\data2.csv')

df_clean = clean_data(df.copy())
df_clean.head()

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

# Features et target
X = df[num_cols]  # tes colonnes numériques
y = df['status']  # remplace par ta colonne à prédire

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Encoder la cible
y_train = y_train.map({'negative': 0, 'positive': 1})
y_test = y_test.map({'negative': 0, 'positive': 1})
# Préprocesseur
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols)
])

# Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

# GridSearch
param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [None, 10, 20],
    'model__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

# Train
grid_search.fit(X_train, y_train)

# Résultats
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validated MAE:", -grid_search.best_score_)

# Évaluation sur test set
y_pred = grid_search.best_estimator_.predict(X_test)
print("Test MAE:", mean_absolute_error(y_test, y_pred))
print("Test R2:", r2_score(y_test, y_pred))


Best Parameters: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__n_estimators': 200}
Best Cross-Validated MAE: 0.01743127962085308
Test MAE: 0.03376893939393939
Test R2: 0.8976308449249834
